In [163]:
# стандартное название для импорта numpy - np
import numpy as np
import pandas as pd
import math


In [216]:
data = pd.read_csv('data.csv', delimiter=', ')
kNN = 4
myUserIndex = 22
# дропаем юзеров, они не нужны, индексы справятся и так
data = data.drop(columns=[data.columns[0]])

print(data)

    Movie 1  Movie 2  Movie 3  Movie 4  Movie 5  Movie 6  Movie 7  Movie 8  \
0        -1        4        1        5       -1       -1       -1        1   
1         4        5        2        3        3        4       -1        5   
2         4        2        5        4        1        5       -1        3   
3         5       -1        2       -1       -1        2        4        3   
4         4        3        3        3        5        3        3        4   
5         2       -1        3        3        3        3        4        2   
6         4        1        4        4        1        4        5        2   
7        -1        5        5        3        3        3        3        3   
8         1        2        3        1        5        3        3        1   
9         5       -1        1       -1        3        3        4        1   
10       -1        5       -1        1        5        2        4        2   
11        5        4        4       -1        3       -1        

<ipython-input-216-03cdf1b0338f>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv('data.csv', delimiter=', ')


In [210]:
# тут будем собирать sim
sim = dict(zip(list(data.index.values) , [0 for i in range(len(data))]))
del sim[myUserIndex]
print(sim)

{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0}


In [211]:
for user in sim.keys():
    set = data.iloc[[user, myUserIndex]]
    set = set.transpose()
    set = set[(set[user] != -1) & (set[myUserIndex] != -1 )]
    sumuv = 0
    sumuu = 0
    sumvv = 0
    for index, row in set.iterrows():
        sumuv +=row[user] * row[myUserIndex]
        sumuu +=row[user] * row[user]
        sumvv +=row[myUserIndex] * row[myUserIndex]
    sim[user] = round(sumuv / (math.sqrt(sumuu) * math.sqrt(sumvv)), 3)
    
closestSim = dict(sorted(list(sim.items()), key=lambda x: x[1], reverse=True)[:kNN])

In [212]:
def tupleToMean(x):
    l = np.array(list(filter(lambda x: x != -1, x)))
    user = l[0]
    l = np.delete(l, [0])
    return [user, l.mean()]

means = dict(list(map(tupleToMean, data.iloc[[*list(closestSim.keys()), myUserIndex]].to_records())))
print(means)
print(closestSim)


{26: 3.3333333333333335, 3: 3.142857142857143, 19: 3.1818181818181817, 4: 3.3076923076923075, 22: 3.391304347826087}
{26: 0.916, 3: 0.903, 19: 0.888, 4: 0.885}


In [214]:
# неоцененные фильмы
unrated = list(data.loc[myUserIndex][data.loc[myUserIndex] == -1].keys())
print(unrated)

['Movie 3', 'Movie 6', 'Movie 7', 'Movie 9', 'Movie 21', 'Movie 22', 'Movie 26']


In [217]:
newRates = {}
for film in unrated:
    sumt = 0
    sumb = 0
    for user in closestSim.keys():
        rate = data.loc[user, film]
        if rate == -1:
            continue 
        sumb += sim[user]
        sumt += sim[user] * (data.loc[user, film] - means[user])
    newRates[film] = round(means[myUserIndex] + sumt/sumb, 3)
print(newRates)

{'Movie 3': 2.651, 'Movie 6': 3.474, 'Movie 7': 2.907, 'Movie 9': 3.144, 'Movie 21': 4.084, 'Movie 22': 3.144, 'Movie 26': 3.898}
